# A Start

It's a modified Dijkstras that utilizes a heuristic function that uses the cost it takes to get to the end node in addition to the cost to transfer to the next node to put back into the priority queue.

In [53]:
# import priority_queue
from queue import PriorityQueue


def manhattan_distance(start, end):
    return abs(start[0] - end[0]) + abs(start[1] - end[1])


def euclidean_distance(start, end):
    return ((start[0] - end[0])**2 + (start[1] - end[1])**2)**0.5


def neighbors(graph, node):
    rows, cols = len(graph), len(graph[0])
    neighbors = []

    if node[0] > 0:
        neighbors.append((node[0]-1, node[1]))
    if node[0] < rows-1:
        neighbors.append((node[0]+1, node[1]))
    if node[1] > 0:
        neighbors.append((node[0], node[1]-1))
    if node[1] < cols-1:
        neighbors.append((node[0], node[1]+1))
    
    return neighbors


def hash_node(node):
    return str(node[0]) + ',' + str(node[1])


def get_tile_cost(graph, node):
    tile = graph[node[0]][node[1]]
    if tile == 'o':
        return 5
    else:
        return 1
    

def draw_grid(grid):
    for row in grid:
        for col in row:
            print(col, end='  ')
        print()


def shortest_path(graph: list, start: list, end: list):
    
    # priority queue used to store the next node to visit
    pq = PriorityQueue()
    
    # set is used to store the nodes that have been visited
    visited = set()
    
    # dictionary used to store the distance from start to each node
    distance = {}
    
    # dictionary used to store the parent of each node to reconstruct the path
    parents = {}
    
    # set the distance of the start node to 0 and initialize the start node in the priority queue
    pq.put((0, start))
    distance[hash_node(start)] = 0
    
    # while there is still nodes to visit
    while not pq.empty():
        cost, node = pq.get()
        
        node_hash = hash_node(node)

        # if node is the end node, return the cost and path
        if node_hash == hash_node(end):
            
            # using the parents dictionary, backtrack from end node to start node
            path = []
            current = end
            while current != start:
                path.append(current)
                current = parents[hash_node(current)]
            path.append(start)

            # return the cost and path. Path is reversed because we backtracked from end to start
            return cost, path[::-1]

        # if node has not been visited
        if node_hash not in visited:

            # add node to visited set
            visited.add(node_hash)
            
            # if node is not in the distance dictionary, set it to infinity
            if node_hash not in distance:
                distance[node_hash] = float('inf')
            
            # optimization step: if the cost to get to the node is greater than the current cost, skip
            # if distance[hash_node(node)] < cost:
            #     continue

            # for each neighbor of the node, if not visited, compute the new distance
            for neighbor in neighbors(graph, node):
                neighbor_hash = hash_node(neighbor)
                
                if neighbor_hash not in visited and graph[neighbor[0]][neighbor[1]] != 'w':
                    
                    if neighbor_hash not in distance:
                        distance[neighbor_hash] = float('inf')
                    
                    # get the cost of the neighbor. If its 'o', then the cost is 5, otherwise its 1
                    # compute the new distance from start to neighbor
                    new_cost = cost + get_tile_cost(graph, neighbor)
                    
                    # if the new distance is less than the current distance
                    if new_cost < distance[neighbor_hash]:
                        
                        # update the new shorter distance
                        distance[neighbor_hash] = new_cost
                        
                        # compute the priority of the neighbor. This is what makes A* different from Dijkstra's
                        priority = new_cost + euclidean_distance(neighbor, end)
                        
                        # add the neighbor to the priority queue with the new distance
                        pq.put((priority, neighbor))
                        
                        # add the parent of the neighbor to the parents dictionary
                        parents[neighbor_hash] = node

    # if no path is found, return None
    return None

graph = [
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'o', ' ', 'e'],
    [' ', ' ', ' ', ' ', ' ', ' ', ' ', 'w', ' ', ' '],
    [' ', ' ', 'o', 'o', 'o', ' ', 'o', 'w', ' ', ' '],
    [' ', 'o', 'o', 'o', 'o', ' ', 'o', 'w', ' ', ' '],
    [' ', 'o', 'o', 'o', 'o', ' ', 'o', 'w', ' ', ' '],
    [' ', 'o', 'o', 'o', 'o', ' ', 'o', 'w', 'o', ' '],
    [' ', ' ', 'o', 'o', 'o', ' ', 'o', 'w', ' ', ' '],
    [' ', ' ', 'o', ' ', ' ', ' ', 'o', 'o', ' ', ' '],
    [' ', ' ', 'o', ' ', 'o', 'o', 'o', 'o', 'o', ' '],
    [' ', ' ', 's', ' ', 'o', 'o', 'o', 'o', 'o', ' '],
]

start = [9, 2]
end = [0, 9]

draw_grid(graph)

path = shortest_path(graph, start, end)

if path is not None:
    cost, path = path

    for node in path:
        if graph[node[0]][node[1]] == 'o':
            graph[node[0]][node[1]] = '*'
        else:
            graph[node[0]][node[1]] = '-'
        
    print('\n\n')
    print('steps: ', len(path)-1)
    print('cost: ', cost)

    draw_grid(graph)

                     o     e  
                     w        
      o  o  o     o  w        
   o  o  o  o     o  w        
   o  o  o  o     o  w        
   o  o  o  o     o  w  o     
      o  o  o     o  w        
      o           o  o        
      o     o  o  o  o  o     
      s     o  o  o  o  o     



steps:  16
cost:  108.72938157766485
                  -  *  -  -  
               -  -  w        
      o  o  o  -  o  w        
   o  o  o  o  -  o  w        
   o  o  o  o  -  o  w        
   o  o  o  o  -  o  w  o     
      o  o  o  -  o  w        
      o  -  -  -  o  o        
      o  -  o  o  o  o  o     
      -  -  o  o  o  o  o     
